In [8]:
from utils.loader import FactoryLoader
from utils.ml import MLPipeline
from utils.preprocessing import PreprocessingFactory
from utils.feature_extraction import *
from utils.utils import *

VAL_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val"
TRAIN_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train"

In [9]:
percent = 100
random = False
batch_size = 48
th = 0.01

# Initialize the FactoryLoader
factory = PreprocessingFactory()
factory.gaussian_smoothing(5)
factory.clahe(clip_limit=1.5)
factory.pad2square(fill=np.nan)
factory.resize((150,150))
factory.hair_removal()
factory.normalize2float()

factory_loader = FactoryLoader(path=VAL_PATH, batch_size=batch_size, factory=factory, percentage=percent, shuffle=random)

# Create the feature extraction pipeline strategy and add desired features
strategy = FeatureExtractionStrategy()

strategy.add_extractor(GradientExtractor(threshold=th))   # Add variance feature

# strategy.add_extractor(MeanExtractor())  # Add mean feature
# strategy.add_extractor(StdExtractor())   # Add standard deviation feature
strategy.add_extractor(VarExtractor(threshold=th))   # Add variance feature
strategy.add_extractor(ColorMomentsExtractor(threshold=th))   # Add color moments feature


# strategy.add_extractor(MeanExtractor("lab"))  # Add mean feature
# strategy.add_extractor(StdExtractor("lab"))   # Add standard deviation feature
strategy.add_extractor(VarExtractor("lab",threshold=th),)   # Add variance feature
strategy.add_extractor(ColorMomentsExtractor("lab",threshold=th),)   # Add color moments feature

# strategy.add_extractor(MeanExtractor("hsv"))  # Add mean feature
# strategy.add_extractor(StdExtractor("hsv"))   # Add standard deviation feature
strategy.add_extractor(VarExtractor("hsv",threshold=th),)   # Add variance feature
strategy.add_extractor(ColorMomentsExtractor("hsv",threshold=th),)   # Add color moments feature

strategy.add_extractor(LBPExtractor(radius=1, n_points=16))
strategy.add_extractor(LBPExtractor(radius=2, n_points=16))
strategy.add_extractor(LBPExtractor(radius=3, n_points=16))
strategy.add_extractor(LBPExtractor(radius=1, n_points=32))
strategy.add_extractor(LBPExtractor(radius=2, n_points=32))
strategy.add_extractor(LBPExtractor(radius=3, n_points=32))
strategy.add_extractor(LBPExtractor(radius=1, n_points=64))
strategy.add_extractor(LBPExtractor(radius=2, n_points=64))
strategy.add_extractor(LBPExtractor(radius=3, n_points=64))

strategy.add_extractor(GLCMExtractor(properties=['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation',]))

pipeline = MLPipeline(dataset_path=TRAIN_PATH, preprocessing_factory=factory, feature_strategy=strategy, classifiers=[], percentage=percent, verbose=True, shuffle=random, batch_size=batch_size)

INFO:utils.ml:MLPipeline initialized with dataset path: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train
INFO:utils.ml:Preprocessing steps


In [10]:
pipeline.run_feature_extraction()

INFO:utils.ml:Running feature extraction...


Processed 5/317 batches.
Processed 10/317 batches.
Processed 15/317 batches.
Processed 20/317 batches.
Processed 25/317 batches.
Processed 30/317 batches.
Processed 35/317 batches.
Processed 40/317 batches.
Processed 45/317 batches.
Processed 50/317 batches.
Processed 55/317 batches.
Processed 60/317 batches.
Processed 65/317 batches.
Processed 70/317 batches.
Processed 75/317 batches.
Processed 80/317 batches.
Processed 85/317 batches.
Processed 90/317 batches.
Processed 95/317 batches.
Processed 100/317 batches.
Processed 105/317 batches.
Processed 110/317 batches.
Processed 115/317 batches.
Processed 120/317 batches.
Processed 125/317 batches.
Processed 130/317 batches.
Processed 135/317 batches.
Processed 140/317 batches.
Processed 145/317 batches.
Processed 150/317 batches.
Processed 155/317 batches.
Processed 160/317 batches.
Processed 165/317 batches.
Processed 170/317 batches.
Processed 175/317 batches.
Processed 180/317 batches.
Processed 185/317 batches.
Processed 190/317 bat

INFO:utils.ml:Feature extraction completed. Extracted 15195 features.


Processed 317/317 batches.


In [11]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier(n_estimators=100)
rf2 = RandomForestClassifier(n_estimators=150)
rf3 = RandomForestClassifier(n_estimators=250)

# Random Forest with Class Weight Balancing
rf4 = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42)
# Random Forest with Feature Selection via max_features
rf5 = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42)
# Random Forest with Min Samples per Leaf
rf6 = RandomForestClassifier(n_estimators=100, min_samples_leaf=5, random_state=42)
# Random Forest with Out-of-Bag Error (OOB)
rf7 = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)
# Random Forest with Bootstrap Disabled
rf8 = RandomForestClassifier(n_estimators=100, bootstrap=False, random_state=42)
# Random Forest with High Number of Trees and Limited Depth
rf9 = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

xgb1 = XGBClassifier(n_estimators=350)
xgb2 = XGBClassifier(n_estimators=450)
xgb3 = XGBClassifier(n_estimators=550)
xgb4 = XGBClassifier(learning_rate=0.05, n_estimators=400, max_depth=3, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb5 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=7, min_child_weight=3, subsample=0.7, colsample_bytree=0.7)
xgb6 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=5, min_child_weight=5, gamma=0.2, subsample=0.8, colsample_bytree=0.8)
xgb7 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=5, min_child_weight=1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=0.1)
xgb8 = XGBClassifier(learning_rate=0.05, n_estimators=500, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb9 = XGBClassifier(learning_rate=0.05, n_estimators=400, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb10 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb11 = XGBClassifier(learning_rate=0.1, n_estimators=500, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)


pipeline.classifiers = [rf1, rf2, rf3, rf4, rf5, rf6, rf7, rf8, rf9,
                        # svm1, svm2, svm3, svm6, svm7, svm8,
                        xgb1, xgb2, xgb3, xgb4, xgb5, xgb6, xgb7, xgb8, xgb9, xgb10, xgb11]
pipeline.fitted_classifiers = {}

In [12]:
pipeline.fit_classifiers()

INFO:utils.ml:Fitting classifiers...
INFO:utils.ml:Fitting classifier: RandomForestClassifier0
INFO:utils.ml:Top 10 features for RandomForestClassifier0: [('lbp_rad1_bins32_10', 0.026613912249443337), ('lbp_rad2_bins64_10', 0.02135175244094795), ('lbp_rad1_bins64_10', 0.020144545957936952), ('color_moments_lab_channel_2_std', 0.016198415620978665), ('lbp_rad1_bins16_10', 0.014669654444594544), ('lbp_rad1_bins64_63', 0.013810101687999832), ('lbp_rad1_bins64_41', 0.011224890345598522), ('color_moments_rgb_channel_0_mean', 0.010500515838116562), ('gradient_magnitude_mean', 0.010115237615624884), ('color_moments_hsv_channel_0_std', 0.009766639931788675)]
INFO:utils.ml:Fitted classifier: RandomForestClassifier0; Done in 17.668471574783325 seconds
INFO:utils.ml:Fitting classifier: RandomForestClassifier1
INFO:utils.ml:Top 10 features for RandomForestClassifier1: [('lbp_rad1_bins64_10', 0.02600589742103939), ('lbp_rad1_bins32_10', 0.024489723285152618), ('lbp_rad1_bins16_10', 0.02123546232826

In [13]:
pipeline.predict_with_classifiers(VAL_PATH)

INFO:utils.ml:Predicting with classifiers on dataset: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val


Processed 5/80 batches.
Processed 10/80 batches.
Processed 15/80 batches.
Processed 20/80 batches.
Processed 25/80 batches.
Processed 30/80 batches.
Processed 35/80 batches.
Processed 40/80 batches.
Processed 45/80 batches.
Processed 50/80 batches.
Processed 55/80 batches.
Processed 60/80 batches.
Processed 65/80 batches.
Processed 70/80 batches.
Processed 75/80 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier0
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier1


Processed 80/80 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier2
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier3
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier4
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier5
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier6
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier7
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier8
INFO:utils.ml:Predictions made with classifier: XGBClassifier9
INFO:utils.ml:Predictions made with classifier: XGBClassifier10
INFO:utils.ml:Predictions made with classifier: XGBClassifier11
INFO:utils.ml:Predictions made with classifier: XGBClassifier12
INFO:utils.ml:Predictions made with classifier: XGBClassifier13
INFO:utils.ml:Predictions made with classifier: XGBClassifier14
INFO:utils.ml:Predictions made with classifier: XGBClassifier15
INFO:utils.ml:Predictions made with classifier: X

{'GT': array([0, 0, 0, ..., 1, 1, 1]),
 'RandomForestClassifier0': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier1': array([0, 0, 1, ..., 0, 0, 1]),
 'RandomForestClassifier2': array([0, 0, 1, ..., 0, 0, 1]),
 'RandomForestClassifier3': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier4': array([0, 0, 1, ..., 0, 0, 1]),
 'RandomForestClassifier5': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier6': array([0, 0, 1, ..., 0, 0, 1]),
 'RandomForestClassifier7': array([0, 0, 1, ..., 0, 0, 1]),
 'RandomForestClassifier8': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier9': array([0, 0, 0, ..., 1, 0, 1]),
 'XGBClassifier10': array([0, 0, 0, ..., 1, 0, 1]),
 'XGBClassifier11': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier12': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier13': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier14': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier15': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier16': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBC

In [14]:
import pandas as pd

df = pd.DataFrame(data=pipeline.calculate_metrics())
df.T

INFO:utils.ml:Metrics for classifier GT: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'kappa': 1.0}
INFO:utils.ml:Metrics for classifier RandomForestClassifier0: {'accuracy': 0.7934668071654373, 'precision': 0.7934256365788157, 'recall': 0.7933493737704145, 'f1': 0.7933795682892224, 'kappa': 0.5867632679324362}
INFO:utils.ml:Metrics for classifier RandomForestClassifier1: {'accuracy': 0.7982086406743941, 'precision': 0.7981558967619886, 'recall': 0.7981201311187719, 'f1': 0.7981360181366401, 'kappa': 0.5962730452801173}
INFO:utils.ml:Metrics for classifier RandomForestClassifier2: {'accuracy': 0.8034773445732349, 'precision': 0.8034494101775089, 'recall': 0.8035370413307361, 'f1': 0.8034555208711595, 'kappa': 0.6069294839531583}
INFO:utils.ml:Metrics for classifier RandomForestClassifier3: {'accuracy': 0.7937302423603794, 'precision': 0.7936662624162625, 'recall': 0.7936999401607469, 'f1': 0.7936804007459322, 'kappa': 0.5873622050086269}
INFO:utils.ml:Metrics for class

cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated

,accuracy,precision,recall,f1,kappa
GT,1.000000,1.000000,1.000000,1.000000,1.000000
RandomForestClassifier0,0.793467,0.793426,0.793349,0.793380,0.586763
RandomForestClassifier1,0.798209,0.798156,0.798120,0.798136,0.596273
RandomForestClassifier2,0.803477,0.803449,0.803537,0.803456,0.606929
RandomForestClassifier3,0.793730,0.793666,0.793700,0.793680,0.587362
RandomForestClassifier4,0.793994,0.793929,0.793959,0.793942,0.587885
RandomForestClassifier5,0.794521,0.794525,0.794614,0.794506,0.589045
RandomForestClassifier6,0.793994,0.793929,0.793959,0.793942,0.587885
RandomForestClassifier7,0.807692,0.807673,0.807561,0.807603,0.615212
RandomForestClassifier8,0.755796,0.755861,0.755928,0.755788,0.511645
